In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
from pathlib import Path
import numpy as np
import tensorflow as tf

def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

def read_tensor_from_image_file(file_name, input_height=299, input_width=299,
				input_mean=0, input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(file_reader, channels = 3,
                                       name='png_reader')
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                  name='gif_reader'))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
  else:
    image_reader = tf.image.decode_jpeg(file_reader, channels = 3,
                                        name='jpeg_reader')
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)

  return result

def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

directory_in_str = "/home/hacker1/images-test/"
pathlist = Path(directory_in_str).glob('**/*.jp*')
for path in pathlist:
    # because path is object not string
    file_name = str(path)
    #im = Image.open(im_pth)

    #file_name = directory_in_str + "axes/1312735_915_main.jpg"
    model_file = "/tmp/output_graph.pb"
    label_file = "/tmp/output_labels.txt"
    input_height = 299
    input_width = 299
    input_mean = 128
    input_std = 128
    input_layer = "Mul"
    output_layer = "final_result"

    graph = load_graph(model_file)
    t = read_tensor_from_image_file(file_name,
                                    input_height=input_height,
                                    input_width=input_width,
                                    input_mean=input_mean,
                                    input_std=input_std)

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name);
    output_operation = graph.get_operation_by_name(output_name);

    with tf.Session(graph=graph) as sess:
        results = sess.run(output_operation.outputs[0],
                            {input_operation.outputs[0]: t})
    results = np.squeeze(results)

    top_k = results.argsort()[-5:][::-1]
    #print(top_k[0])
    labels = load_labels(label_file)
    #for i in top_k:
    if labels[top_k[0]] not in file_name:
        print(file_name, labels[top_k[0]], results[top_k[0]], "\n")

/home/hacker1/images-test/pulleys/100517.jpeg carabiners 0.585958 

/home/hacker1/images-test/hardshell_jackets/1286053_431_main.jpg hardshell jackets 0.905231 

/home/hacker1/images-test/hardshell_jackets/10116633x1072823_zm.jpeg hardshell jackets 0.956507 

/home/hacker1/images-test/hardshell_jackets/1282013_405_main.jpg hardshell jackets 0.948951 

/home/hacker1/images-test/hardshell_jackets/2028652_408_main.jpg hardshell jackets 0.986559 

/home/hacker1/images-test/hardshell_jackets/2008046_601_main.jpg insulated jackets 0.533285 

/home/hacker1/images-test/hardshell_jackets/10048480x1058000_zm.jpeg insulated jackets 0.557995 

/home/hacker1/images-test/hardshell_jackets/10116633x1012905_zm.jpeg hardshell jackets 0.867438 

/home/hacker1/images-test/hardshell_jackets/1281947_001_main.jpg hardshell jackets 0.946636 

/home/hacker1/images-test/hardshell_jackets/10048480x1045978_zm.jpeg insulated jackets 0.7489 

/home/hacker1/images-test/hardshell_jackets/2027536_408_main.jpg insulat